# Implementation of Kalman Filter with Python Language 

source: https://arxiv.org/pdf/1204.0375.pdf

## 1. Mathematical formulation

Given measurements $y \in \Re^m$, we want to estimate the state $x \in \Re ^n$ of a discrete time controlled process that is governed by the linear stochastic equation:

$$ x_k = Ax_{k-1} + Bu_k + w_{k} $$

where

* $A$ is the state transition model which is applied to the previous state $x_{k-1}$
* $B$ is the control-input model which is applied to the control vector $u_k$
* $w_k$ is the process noise which is assumed to be drawn from a zero mean multivariate normal distribution, $\mathcal{N}$, with covariance $Q_k$: therefore $w_k \sim \mathcal{N}(0,\,Q_k)$.

At time k a measurement (or observation) $y_k$ of the true state $x_k$ is made according to

$$y_k = Hx_k + v_k $$

where

* $H$ is the observation model which maps the true state space into the observed space

* $v_k$ is the observation noise which is assumed to be zero mean Gaussian white noise with covariance $R_k$: therefore $v_k \sim \mathcal{N}(0,\,R_k)$

###  Comments

1. The mutual state, and the noise vectors at each step are all assumed to be mutually independent $ \{x_0, w_1, \dots, w_k, v_1, \dots, v_k\}  $

2. Read about **unmodelled dynamics**...


## 2. Kalman steps

The Kalman filter process has two steps

1. **prediction step** -- the next state of the system is predicted given the previous measurements

2. **update step** -- the current state of the system is estimated given the measurement at that time step

... check the equations...

## 3. Implementation

In [56]:
from numpy import dot, sum, tile, linalg, log, pi, exp
from numpy.linalg import inv, det
from numpy.random import randn


In [57]:
def predict(X, P, A, Q, B, U):
    X = dot(A, X) + dot(B, U)
    P = dot(A, dot(P, A.T)) + Q
    return (X, P)


def update(X, P, Y, H, R):
    IM = dot(H, X)
    IS = R + dot(H, dot(P, H.T))
    K = dot(P, dot(H.T, inv(IS)))
    X = X + dot(K, (Y - IM))
    P = P - dot(K, dot(IS, K.T))
    LH = gauss_pdf(Y, IM, IS)
    return (X, P, K, IM, IS, LH)


def gauss_pdf(X, M, S): 
    if M.shape[1] == 1:
        DX = X - tile(M, X.shape[1])
        E = 0.5 * sum(DX * (dot(inv(S), DX)), axis=0)
        E = E + 0.5 * M.shape[0] * log(2 * pi) + 0.5 * log(det(S))
        P = exp(-E)
        
    elif X.shape[1] == 1:
        DX = tile(X, M.shape()[1])- M
        E = 0.5 * sum(DX * (dot(inv(S), DX)), axis=0)
        E = E + 0.5 * M.shape[0] * log(2 * pi) + 0.5 * log(det(S))
        P = exp(-E)
        
    else:
        DX = X-M
        E = 0.5 * dot(DX.T, dot(inv(S), DX))
        E = E + 0.5 * M.shape[0] * log(2 * pi) + 0.5 * log(det(S))
        P = exp(-E)
        
    return (P[0], E[0]) 

## 4. Application

In [58]:
import numpy as np


In [61]:
randn = np.random.randn

# timestep
dt = .1

# initialization
X = np.array([[0.0], [0.0], [0.1], [0.1]])
P = np.diag((0.01, 0.01, 0.01, 0.01))
A = np.array([[1, 0, dt , 0], [0, 1, 0, dt], [0, 0, 1, 0], [0, 0, 0, 1]])
Q = np.eye(X.shape[0])
B = np.eye(X.shape[0])
U = np.zeros((X.shape[0],1))

# Measurement matrices
Y = np.array([[X[0,0] + abs(randn(1)[0])], [X[1,0] + abs(randn(1)[0])]])
H = np.array([[1, 0, 0, 0], [0, 1, 0, 0]])
R = np.eye(Y.shape[0])

# Number of iterations in Kalman Filter
N_iter = 50

# Applying the Kalman Filter
for i in np.arange(0, N_iter):
    (X, P) = predict(X, P, A, Q, B, U)
    (X, P, K, IM, IS, LH) = update(X, P, Y, H, R)
    Y = np.array([[X[0,0] + abs(0.1 * randn(1)[0])],[X[1, 0] + abs(0.1 * randn(1)[0])]]) 